# Laboratorio 5: Programación Lineal Mixta (MA4702, otoño 2023)

Fecha: 28 de junio, 2023.<br/>
Profesor: Marcos Kiwi.<br/>
Auxiliares: Benjamín Jauregui y Cristián Palma.<br/>

## Preliminares

Este laboratorio consta de 2 etapas. Una Tarea Introductoria (TI) y una etapa de Trabajo Presencial (TP). La nota global del laboratorio será 40 % (TI) + 60 % (TP).

La tarea introductoria es de carácter estrictamente individual. La etapa presencial es de carácter grupal. 


## Laboratorio 5 Tarea Introductoria (TI)

## Instrucciones

* La respuesta a esta tarea consiste en completar este mismo archivo `.ipnyb` y entregarlo a través de Tareas en UCursos, antes de las **14:15 del miércoles 28 de junio de 2023** (Se descontará 1.0 pts. por cada 15 minutos o fracción de atraso.)

* Realice esta primera tarea de manera **individual**. Bajo ningún motivo discuta sus respuestas, ni comparta o deje a disposición los archivos con sus compañeros. Se hará un estudio de similitudes entre los archivos resultantes para detectar posibles plagios.

<div class="alert alert-block alert-warning">
    Advertencia: Si en el archivo que entregue alguna de las celdas arroja un mensaje de error al ejecutar, o el código entra en un loop infinito y no retorna con ninguna salida, Ud. deberá agregar una nota al respecto, explicando cuál cree que es el problema. Si no lo hace, se le penalizará adicionalmente.               
</div>


## Datos personales

<div class="alert alert-block alert-success">
    Indique su nombre: <br/>              
</div>


## Introducción: Descomposición de Bender

El objetivo de esta tarea es familiarizarse con la descomposición de Bender. Esta descomposición consiste esencialmente en aplicar descomposición de Dantzig-Wolfe en el programa dual, pero tiene características que hacen útil considerarla por separado. En esta primera parte del laboratorio discutiremos un caso particular de la descomposición de Bender (el caso de un único sub-problema), y en la parte presencial del laboratorio abordaremos el caso general (el caso de múltiples sub-problemas). 

Consideremos el siguiente tipo de programa lineal mixto:
$$
\begin{array}{rlll}
(PLM):\ z & =\max_{x, y} c^Tx + d^Ty \\
\text{s.a.} & Ax+By \leq b, \\
& y \geq \vec{0}\\
& x\ \text{binaria}, 
\end{array}
$$

el cual reformulamos de la siguiente forma:

$$
\begin{array}{rlll}
z & = \max_{x} c^Tx + z(x) \\
\text{s.a.} & x\ \text{binaria}, 
\end{array} 
$$

donde

$$
\begin{array}{rlll}
z(x) & = \max_{y} d^Ty \\
\text{s.a.} & By\leq b-Ax\\
& y\geq\vec{0}.
\end{array}
$$

Dualizando, obtenemos:

$$
\begin{array}{rlll}
(SubP_x):\ z(x) & = \min_{w} w^T(b-Ax) \\
\text{s.a.} & w^TB \geq d, \\
& w\geq\vec{0}.
\end{array}
$$

Definimos $\mathcal{P}=\{w \mid w^TB \geq d\}$. Asumiendo que $\mathcal{P}$ es puntiagudo, por Minkowski-Weyl, sigue que $\mathcal{P}=\mathrm{conv}(\mathcal{V})+\mathrm{cono}(\mathcal{R})$ donde $V$ es conjunto finito de puntos extremos y $\mathcal{R}$ es conjunto finito de rayos extremos de $\mathcal{P}$, respectivamente. Recordando que

$$
\begin{align*}
z(x) > -\infty \iff r^T(b-Ax)\geq \vec{0}, \qquad \forall r\in\mathcal{R},
\end{align*}
$$

y que si $z(x)>-\infty$, entonces $z(x) = \min_{v\in\mathcal{V}} v^T(b-Ax)$ (equivalentemente, $\zeta=z(x)$ es el mayor valor tal que  $v^T(b-Ax) \geq \zeta$, $\forall v\in\mathcal{V}$), 
sigue que

$$
\begin{array}{rlll}
(PLM'):\ z & = \max_{x} c^Tx+\zeta \\
\ \ \text{s.a.}
& v^T(b-Ax) \geq \zeta, & \forall v\in\mathcal{V}, \\
& r^T(b-Ax) \geq \vec{0}, & \forall r\in\mathcal{R}, \\
& x\ \text{binaria}.
\end{array}
$$

La discusión anterior motiva el siguiente procedimiento iterativo para resolver $(PLM)$:

> **Descomoposición de Bender para problemas en dos etapas.**
> 1. Definir el *master problem* inicial $(MP)$ versión relajada de $(PLM')$ pero donde se han omitido gran parte de sus restricciones (y/o reemplazado por desigualdades válidas).
> 2. Obtener $(x^\star,z^\star)$ solución óptima de $(MP)$.
> 3. Si $(SubP_{x^\star})$ es factible y su valor óptimo es al menos $z^\star$, entonces $x^\star$ es solución óptima. Reemplazar en $(PLM)$, determinar el correspondiente valor de $y^\star$ y **parar**.
> 4. Si $(SubP_{x^\star})$ tiene solución óptima cuyo valor es menor a $z^\star$, identificar una s.b.f. $v$ de $(SubP_{x^\star})$, agregar la restricción $v^T(b-Ax)\geq \zeta$ a la instancia actual de $(MP)$, e ir a 2.
> 5. Si $(SubP_{x^\star})$ es no acotado, identificar un rayo extremo $r$ de $(Sub_{x^\star})$ de costo positivo, agregar la restricción $r^T(b-Ax)\geq \vec{0}$ a la instancia actual de $(MP)$, e ir a 2.

<div class="alert alert-warning">
    Referencias: Bertsimas, Tsitsiklis, Introduction to Linear Optimization, Seccón 6.5, pág. 254-260, Athena Scientific, 1997.
</div>

## Ejercicio 1

El propósito de este ejercicio es resolver el siguiente programa lineal mixto usando la descomposición de Bender descrita en la introducción:

$$
\begin{array}{c}
& 
\begin{array}{rrrrrrcr}
z & =\max_{x,y} -15x_1 & -10x_2 & +8y_1 & +9y_2 & +5y_3 & +6y_4 \\
\text{s.a.} & -x_1 & -x_2 & & & & & \leq & -1 \\
&&& y_1 & & +y_3 & &  \leq & 1, \\
&&& y_1& & & +y_4 &  \leq & 1, \\
&&& & y_2 & +y_3 & &  \leq &  1, \\
&&& & y_2 & & +y_4 &  \leq &  1, \\ 
 & -x_1 & & y_1 & & & & \leq & 0, \\
 & -x_1 & & & y_2 & & & \leq & 0, \\
 & & -x_2 & & & y_3 & & \leq & 0, \\
 & & -x_2 & & & & y_4 & \leq & 0, \\
\end{array}
\\[2pt]
& y_1, y_2, y_3, y_4 \in\mathbb{R}_+, \\
& x_1, x_2 \in \{0,1\}.
\end{array}
$$

Claramente, $z\leq 28$. Considere el siguiente *master problem* inicial:


$$
\begin{array}{rlll}
(MP):\ z & =\max_{x,\zeta} -15x_1 - 10x_2 + \zeta \\
\text{s.a.} & \zeta \leq 28, \\
& x\in [0,1]^2, \\ 
& \eta\in\mathbb{R}.
\end{array}
$$

Complete la celda subsiguiente, realizando hasta un máximo de 8 iteraciones o hasta obtener el óptimo. En cada iteración deberá resolver una instancia de $(MP)$ y un $(SubP_x)$ (a los correspondientes modelos llámelos `mstr` y `subProb`, respectivamente). En cada iteración, después de resolver `mstr` o `subProb`, invoque la función `muestra(mdl)` que se provee en la siguiente celda, reemplazando `mdl` por `mstr` o por `subProb` según corresponda. 

> **Nota:** Cuando se resuelve un modelo no acotado en `JuMP/Gurobi`, los valores que el `Gurobi` le asigna a las variables determinan un rayo extremo de costo positivo (respectivamente, negativo) en el caso de problemas de maximización (respectivamente, minimización), **salvo** cuando el pre-procesamiento inicial que `Gurobi` realiza determine por si solo que el modelo es no acotado. Una forma de forzar que `Gurobi` siempre retorne un rayo extremo cuando el modelo es infactible es desactivar el pre-procesamiento fijando como atributo del modelo `"Presolve" => 0`.   


In [26]:
using JuMP, Gurobi;           

const GUROBI_ENV = Gurobi.Env() 

function muestra( mdl )
    # Recibe un modelo que ha sido resuelto e imprime su status, valor objetivo, y valor de sus variables.
    println( mdl )
    st = termination_status( mdl )
    println( st )
    if st == MOI.OPTIMAL
        println( objective_value( mdl ) )
    end
    varArray = all_variables(mdl)
    for i = 1:size(varArray,1)
        println( varArray[i], " = ", value(varArray[i]) )
    end
    println()
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-16


muestra (generic function with 1 method)

In [46]:
A = [-1 -1; 0 0; 0 0; 0 0; 0 0; -1 0; -1 0; 0 -1; 0 -1]
B = [0 0 0 0; 1 0 1 0; 1 0 0 1; 0 1 1 0; 0 1 0 1; 1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1]
b = [-1 1 1 1 1 0 0 0 0]'
c = [-15 -10]'
d = [8 9 5 6]'

println( "======= Iter 0 ======" ); 
println( "======= Master ======" ); 
mstr = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "OutputFlag" => 0));
@variable( mstr, z )
@variable( mstr, 1 >= x[1:size(c,1)] >= 0 )
@constraint( mstr, mstrCon, z <= sum(d) )
@objective( mstr, Max, z + sum(c.*x) )
optimize!( mstr )
muestra( mstr )

println( "===== SubProb =====" ); 
subProb = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "OutputFlag" => 0, "Presolve" => 0 ));

@variable(subProb, w[1:length(b)] >= 0)
@constraint(subProb, [i in 1:length(d)], sum(w.*B[1:end, i]) ≥ d[i])
@objective(subProb, Min, sum(w .* (b .- A*value.(x))))

optimize!(subProb)

muestra( subProb ) 


maxIter = 8

for i in 1:maxIter
    println( "===== Iter $i ======" ); 
    println( "===== Master ======" );

    muestra( mstr )

    println( "===== SubProb =====" ); 

    muestra( subProb )

    ζ = value(z)

    if termination_status(subProb) == MOI.OPTIMAL
        if objective_value(subProb) ≥ ζ
            println("===== Óptimo encontrado =====")
            break
        else
            println("===== Agregando restricción por vértice =====")
            v = value.(w)
            @constraint(mstr, sum(v.*(b .- A*x)) ≥ z)
        end
    else
        println("===== Agregando restricción por rayo =====")
        r = value.(w)
        println(sum(r.*(b .- A*x)))
        @constraint(mstr, sum(r.*(b .- A*x)) ≥ 0)
    end

    optimize!(mstr)
    newxvalue = value.(x)
    @objective(subProb, Min, sum(w .* (b .- A*newxvalue)))

    optimize!(subProb)
end


println( "===== Cálculo y =====")

plm = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "OutputFlag" => 0));
@variable(plm, y[1:length(d)] >=0)
@constraint(plm, [i in 1:length(b)], sum(B[i, 1:end].*y) <= b[i] - (A*value.(x))[i])
@objective(plm, Max, sum(y.*d))
optimize!(plm)

muestra( plm )


======= Iter 0 ======
======= Master ======


Max z - 15 x[1] - 10 x[2]
Subject to
 mstrCon : z ≤ 28.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[1] ≤ 1.0
 x[2] ≤ 1.0

OPTIMAL
28.0
z = 28.0
x[1] = 0.0
x[2] = 0.0

===== SubProb =====
Min -w[1] + w[2] + w[3] + w[4] + w[5]
Subject to
 w[2] + w[3] + w[6] ≥ 8.0
 w[4] + w[5] + w[7] ≥ 9.0
 w[2] + w[4] + w[8] ≥ 5.0
 w[3] + w[5] + w[9] ≥ 6.0
 w[1] ≥ 0.0
 w[2] ≥ 0.0
 w[3] ≥ 0.0
 w[4] ≥ 0.0
 w[5] ≥ 0.0
 w[6] ≥ 0.0
 w[7] ≥ 0.0
 w[8] ≥ 0.0
 w[9] ≥ 0.0

DUAL_INFEASIBLE
w[1] = 1.0
w[2] = 0.0
w[3] = 0.0
w[4] = 0.0
w[5] = 0.0
w[6] = 0.0
w[7] = 0.0
w[8] = 0.0
w[9] = 0.0

===== Iter 1 ======
===== Master ======
Max z - 15 x[1] - 10 x[2]
Subject to
 mstrCon : z ≤ 28.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[1] ≤ 1.0
 x[2] ≤ 1.0

OPTIMAL
28.0
z = 28.0
x[1] = 0.0
x[2] = 0.0

===== SubProb =====
Min -w[1] + w[2] + w[3] + w[4] + w[5]
Subject to
 w[2] + w[3] + w[6] ≥ 8.0
 w[4] + w[5] + w[7] ≥ 9.0
 w[2] + w[4] + w[8] ≥ 5.0
 w[3] + w[5] + w[9] ≥ 6.0
 w[1] ≥ 0.0
 w[2] ≥ 0.0
 w[3] ≥ 0.0
 w[4] ≥ 0.0
 w[5] ≥ 0.0
 w[6] ≥ 0.0
 w[7] ≥ 0.0
 w[


===== Master ======
Max z - 15 x[1] - 10 x[2]
Subject to
 x[1] + x[2] ≥ 1.0
 -z + 17 x[1] + 11 x[2] ≥ 0.0
 -z + 3 x[1] ≥ -14.0
 mstrCon : z ≤ 28.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[1] ≤ 1.0
 x[2] ≤ 1.0

OPTIMAL
2.0
z = 17.0
x[1] = 1.0
x[2] = 0.0

===== SubProb =====
Min w[2] + w[3] + w[4] + w[5] + w[6] + w[7]
Subject to
 w[2] + w[3] + w[6] ≥ 8.0
 w[4] + w[5] + w[7] ≥ 9.0
 w[2] + w[4] + w[8] ≥ 5.0
 w[3] + w[5] + w[9] ≥ 6.0
 w[1] ≥ 0.0
 w[2] ≥ 0.0
 w[3] ≥ 0.0
 w[4] ≥ 0.0
 w[5] ≥ 0.0
 w[6] ≥ 0.0
 w[7] ≥ 0.0
 w[8] ≥ 0.0
 w[9] ≥ 0.0

OPTIMAL
17.0
w[1] = 0.0
w[2] = 5.0
w[3] = 0.0
w[4] = 0.0
w[5] = 9.0
w[6] = 3.0
w[7] = 0.0
w[8] = 0.0
w[9] = 0.0

===== Óptimo encontrado =====
===== Cálculo y =====


Max 8 y[1] + 9 y[2] + 5 y[3] + 6 y[4]
Subject to
 0 ≤ 0.0
 y[1] + y[3] ≤ 1.0
 y[1] + y[4] ≤ 1.0
 y[2] + y[3] ≤ 1.0
 y[2] + y[4] ≤ 1.0
 y[1] ≤ 1.0
 y[2] ≤ 1.0
 y[3] ≤ 0.0
 y[4] ≤ 0.0
 y[1] ≥ 0.0
 y[2] ≥ 0.0
 y[3] ≥ 0.0
 y[4] ≥ 0.0

OPTIMAL
17.0
y[1] = 1.0
y[2] = 1.0
y[3] = 0.0
y[4] = 0.0

